In [81]:
# Extract SO-CI information from MOLPRO outputs for atoms
#   Read exptl data from Excel file, combine with weights to get E_so
#   The experimental Excel file is generated by get_NIST_atomic_data.ipynb
# Add a fourth way to get E_so:  by fitting term energies to exptl level energies
#   (off-diagonal elements of the spin-orbit matrix are from a Molpro SO-CI calculation)
# KKI 2/5/2024
import re, sys, os
import numpy as np
import pandas as pd
import scipy
#import matplotlib.pyplot as plt
#from collections import Counter
#from sklearn.cluster import KMeans

import chem_subs as chem
import molpro_subs as mpr

pd.set_option('display.max_rows', None)
np.set_printoptions(suppress=True)

### Specify Molpro SO-CI output file

In [82]:
fdir = r'C:\Users\irikura\OneDrive - NIST\Karl\atomic_SOC\calculations\Co_I'
fname = 'co_20Q24D_1710_ctzdk_fc.pro'

In [83]:
fsoc = os.sep.join([fdir, fname])
print(f'Reading MOLPRO file "{fsoc}"')
atom = mpr.stoichiometry(fsoc)
charge = int(np.rint(mpr.total_charge(fsoc, verbose=True)))
print(f'The atom is {atom} with charge {charge}')
# append charge, to match sheet names in exptl data file
if charge > 0: 
    atom += '+'
elif charge < 0:
    atom += '-'
if abs(charge) > 1:
    atom += f'{abs(charge)}'
PG = mpr.read_compgroup(fsoc)
print(f'The computational point group is {PG}')

Reading MOLPRO file "C:\Users\irikura\OneDrive - NIST\Karl\atomic_SOC\calculations\Co_I\co_20Q24D_1710_ctzdk_fc.pro"
The atom is Co with charge 0
The computational point group is Ci


In [84]:
SOCI = mpr.fullmatSOCI(fsoc, atom=True)

Computational group = Ci
CASSCF states:
    24 Doublet
    20 Quartet
---CASSCF state 1.1 matched to MRCI state 2.1
---CASSCF state 1.1 matched to MRCI state 3.1
---CASSCF state 1.1 matched to MRCI state 4.1
---CASSCF state 1.1 matched to MRCI state 5.1
---CASSCF state 1.1 matched to MRCI state 6.1
---CASSCF state 1.1 matched to MRCI state 7.1
---CASSCF state 8.1 matched to MRCI state 9.1
---CASSCF state 8.1 matched to MRCI state 10.1
---CASSCF state 8.1 matched to MRCI state 11.1
---CASSCF state 8.1 matched to MRCI state 12.1
---CASSCF state 8.1 matched to MRCI state 13.1
---CASSCF state 8.1 matched to MRCI state 14.1
---CASSCF state 15.1 matched to MRCI state 16.1
---CASSCF state 15.1 matched to MRCI state 17.1
---CASSCF state 18.1 matched to MRCI state 19.1
---CASSCF state 18.1 matched to MRCI state 20.1
---CASSCF state 10.1 matched to MRCI state 1.1
---CASSCF state 10.1 matched to MRCI state 2.1
---CASSCF state 10.1 matched to MRCI state 3.1
---CASSCF state 10.1 matched to MRCI sta

In [85]:
SOCraw = SOCI.vals.min()
print(f'From lowest level and lowest uncoupled term energy, raw theoretical SOCraw = {SOCraw:.3f} cm-1')

From lowest level and lowest uncoupled term energy, raw theoretical SOCraw = -861.098 cm-1


In [86]:
# Read experimental energy levels
if charge >= 0:
    atstr = atom + '_' + 'I' * (charge + 1)
else:
    # anion
    atstr = atom + '_neg'
fxl = f'{atstr}_exptl_levels.xlsx'
dfexpt = pd.read_excel(fxl)
print(f'Experimental energy levels read from {fxl}')

Experimental energy levels read from Co_I_exptl_levels.xlsx


In [87]:
# Find the number of decimal places in the level energies
Ecol = 'Level (cm-1)'  # the exptl energy column
ndecim = 0
for e in dfexpt[Ecol]:
    words = str(e).split('.')
    # count numeric digits
    n = sum(c.isdigit() for c in words[-1])
    ndecim = max(n, ndecim)
print(f'Experimental energies are provided to {ndecim} decimal digits')

Experimental energies are provided to 2 decimal digits


In [88]:
# display formatting
fmt = {'Eshift': '{:.1f}', 'Pct': '{:.3f}', 'degen': '{:.0f}'}
fmt[Ecol] = '{:.' + str(ndecim) + 'f}'
for col in ['J', 'Ecalc', 'E_dif', 'Erel', 'Eshift', 'err', 'Eterm', 'cm-1', 'fitted',
           'wmean', 'wstds', 'uwmean', 'uwstds']:
    fmt[col] =  fmt['Eshift']
for col in ['dif', 'Theory', 'ecm', 'SOC', 'RMSE']:
    fmt[col] = '{:.2f}'

In [89]:
dfterm = SOCI.average_terms(be_close=['Energy', 'Edav', 'Eref', 'dipZ', 'C0'], always=True)
# drop the dipZ column
dfterm.drop(columns=['dipZ'], inplace=True)
print(f'Averaged terms from MRCI ({fname}):')
dfso = SOCI.assign_atomic_J(quiet=True)  # create SOCI.dfso
Egl = SOCI.dfso.E.min()  # energy of ground level
dfterm['Erel'] = (dfterm.Edav - Egl) * chem.AU2CM
display(dfterm.style.format(fmt))
print('"ecm"  is relative to the lowest term (cm-1)')
print('"Erel" is relative to the ground level')

Averaged terms from MRCI (co_20Q24D_1710_ctzdk_fc.pro):


,Term,Edav,idx,ecm,Erel
0,(1)4F,-1392.556323,[0 1 2 3 4 5 6],0.00,862.4
1,(2)4F,-1392.539358,[ 8 12 7 10 11 13 9],3723.30,4585.7
2,(1)2F,-1392.521050,[20 21 23 24 22 25 26],7741.60,8603.9
3,(1)4P,-1392.495025,[14 15 16],13453.40,14315.8
4,(2)4P,-1392.484394,[19 18 17],15786.60,16649.0
5,(1)2G,-1392.481372,[33 34 32 31 30 29 27 28 35],16449.90,17312.3
6,(1)2P,-1392.471238,[36 38 37],18674.00,19536.4
7,(1)2D,-1392.460655,[41 39 40 43 42],20996.80,21859.1


"ecm"  is relative to the lowest term (cm-1)
"Erel" is relative to the ground level


In [90]:
irreps_ci = set(SOCI.dfci.Irrep)
if (PG == 'Ci') and (len(irreps_ci) == 1):
    if 1 in irreps_ci:
        parity = 'even'
    else:
        parity = 'odd'
else:
    # ask user for parity of interest
    parity = input('Please choose "even" or "odd" parity: ')
print(f'Experimental states will be restricted to parity = {parity}')

Experimental states will be restricted to parity = even


In [91]:
# Delete any ionization limit
ilim = dfexpt[dfexpt.Term == 'Limit'].index.min()
# delete the "Limit" row and everything past it
n1 = len(dfexpt)
dfexpt = dfexpt.truncate(after=ilim-1)
n2 = len(dfexpt)
if n2 < n1:
    print(f'Discarding {n1-n2} ionized or metastable states')
oddstr = '\*$|°' # regex to identify terms of odd parity
dfeven = dfexpt[~(dfexpt.Term.str.contains(oddstr) | dfexpt.Configuration.str.contains(oddstr))].copy()
dfodd = dfexpt[dfexpt.Term.str.contains(oddstr) | dfexpt.Configuration.str.contains(oddstr)].copy()
print(f'{len(dfexpt)} experimental levels ({len(dfeven)} even and {len(dfodd)} odd)')
# Select by parity
if parity == 'even':
    # discard odd levels ('Term' field ends with '*')
    dfexpt = dfeven.copy()
elif parity == 'odd':
    dfexpt = dfodd.copy()
else:
    chem.print_err('', f'Parity of "{parity}" is not recognized')
n3 = len(dfexpt)
print(f'{n3} levels accepted for parity = {parity}')
# Assign unique term symbols
dfexpt = chem.unique_labels_exptl_terms(dfexpt, verbose=True, always=True)
dfexpt

Discarding 1 ionized or metastable states
306 experimental levels (110 even and 196 odd)
110 levels accepted for parity = even


,Configuration,Term,J,Level (cm-1),uTerm
0,3p63d74s2,a 4F,9/2,0.00,(1)4F
1,3p63d74s2,a 4F,7/2,816.00,(1)4F
2,3p63d74s2,a 4F,5/2,1406.84,(1)4F
3,3p63d74s2,a 4F,3/2,1809.33,(1)4F
4,3p63d8(3F)4s,b 4F,9/2,3482.82,(2)4F
5,3p63d8(3F)4s,b 4F,7/2,4142.66,(2)4F
6,3p63d8(3F)4s,b 4F,5/2,4690.18,(2)4F
7,3p63d8(3F)4s,b 4F,3/2,5075.83,(2)4F
8,3p63d8(3F)4s,a 2F,7/2,7442.41,(1)2F
9,3p63d8(3F)4s,a 2F,5/2,8460.81,(1)2F


In [92]:
def match_expt_theory_simple(dfexpt, dftheory):
    # Match exptl and theoretical levels, based upon leading term
    #    (will fail if term assignments differ)
    # Return a DataFrame containing both theory and expt
    if 'degen' not in dfexpt.columns:
        # Add degeneracy
        dfexpt['degen'] = (2 * dfexpt.J.apply(chem.halves_to_float)) + 1
    dfcomp = dfexpt.copy()
    dfcomp['Tcalc'] = ''  # term assignment in computation
    dfcomp['termwt'] = ''
    dfcomp['Ecalc'] = np.nan
    for i, row in dftheory.iterrows():
        term = row.Lead
        J = chem.halves_to_float(row.J)
        for ix, rowx in dfexpt.iterrows():
            xTerm = rowx.uTerm
            xJ = chem.halves_to_float(rowx.J)
            if (xTerm == term) and (xJ == J):
                if not np.isnan(dfcomp.at[ix, 'Ecalc']):
                    print('Already paired!', display(rowx.to_frame().T))                   
                else:
                    dfcomp.at[ix, 'Ecalc'] = row.Erel
                    dfcomp.at[ix, 'Tcalc'] = term
                    dfcomp.at[ix, 'termwt'] = row.termwt
    dfcomp['err'] = dfcomp.Ecalc - dfcomp[Ecol]
    return dfcomp

In [93]:
dfdiff = match_expt_theory_simple(dfexpt, dfso)
# Convert str values of J to float
dfdiff['J'] = dfdiff.J.apply(chem.halves_to_float)
warnThresh = 1000  # highlight errors larger than this (cm-1)
# drop rows with NaN (no matching level in the calculation)
dfdiff.dropna(axis=0, inplace=True)
print(f'Please inspect the following pairing of theory ("Ecalc") with expt ("{Ecol}")')
print('Disagreements in term assignments are highlighted in red')
print(f'Errors > {warnThresh} cm-1 are highlighted in yellow')
selcols = ['Configuration', 'uTerm', 'J', Ecol, 'Tcalc', 'Ecalc', 'err']
display(dfdiff[selcols].style.apply(lambda x: ["background: yellow" if abs(v) > warnThresh else "" for v in x], 
              subset=pd.IndexSlice[['err']]).apply(lambda x: (x != dfdiff['uTerm']).map({True: "background-color: red; \
              color: white", False: ""}), subset=['Tcalc']).format(fmt))

Please inspect the following pairing of theory ("Ecalc") with expt ("Level (cm-1)")
Disagreements in term assignments are highlighted in red
Errors > 1000 cm-1 are highlighted in yellow


,Configuration,uTerm,J,Level (cm-1),Tcalc,Ecalc,err
0,3p63d74s2,(1)4F,4.5,0.00,(1)4F,0.0,0.0
1,3p63d74s2,(1)4F,3.5,816.00,(1)4F,851.9,35.9
2,3p63d74s2,(1)4F,2.5,1406.84,(1)4F,1474.4,67.5
3,3p63d74s2,(1)4F,1.5,1809.33,(1)4F,1898.5,89.1
4,3p63d8(3F)4s,(2)4F,4.5,3482.82,(2)4F,3838.8,356.0
5,3p63d8(3F)4s,(2)4F,3.5,4142.66,(2)4F,4536.0,393.3
6,3p63d8(3F)4s,(2)4F,2.5,4690.18,(2)4F,5133.8,443.6
7,3p63d8(3F)4s,(2)4F,1.5,5075.83,(2)4F,5579.2,503.4
8,3p63d8(3F)4s,(1)2F,3.5,7442.41,(1)2F,8162.8,720.4
9,3p63d8(3F)4s,(1)2F,2.5,8460.81,(1)2F,9288.4,827.6


In [94]:
# No theoretical calculations are needed to use eq. (1)
xterms = []  # list of term labels
eterms = []  # list of term energies
for term in dfdiff.uTerm:
    if term not in xterms:
        xterms.append(term)
for Term in xterms:
    subdf = dfexpt[dfexpt.uTerm == Term]
    emean = np.dot(subdf.degen, subdf[Ecol]) / subdf.degen.sum()
    eterms.append(emean)
dfeq1 = pd.DataFrame({'Term': xterms, 'Eterm': eterms}).sort_values('Eterm').reset_index(drop=True)
print('Term energies (cm-1) using eq. (1) (experimental data with naive model)')
display(dfeq1.style.format(fmt))
SOC1 = -1 * np.round(dfeq1.at[0, 'Eterm'], 3)
lowterm = dfeq1.at[0, 'Term']
print(f'The term of lowest energy is \t{lowterm} \twith SOC1 = {SOC1} cm-1')
levterm = dfexpt.uTerm.values[0]

target = levterm

if levterm != lowterm:
    # The lowest term is not the leading term in the lowest level
    SOC1alt = SOC1
    SOC1 = -1 * np.round(dfeq1[dfeq1.Term == levterm]['Eterm'].values[0], 3)
    print(f'The lowest level belongs to \t{levterm} \twith SOC1 = {SOC1} cm-1')
print()
print(f'Term {target} is selected for further SOC calculation')
print('    to change this, assign the variable "target" to another term in the table above')

Term energies (cm-1) using eq. (1) (experimental data with naive model)


,Term,Eterm
0,(1)4F,793.1
1,(2)4F,4157.6
2,(1)2F,7878.9
3,(1)4P,13976.4
4,(2)4P,15549.3
5,(1)2D,16655.1
6,(1)2G,16808.2
7,(1)2P,18518.0


The term of lowest energy is 	(1)4F 	with SOC1 = -793.084 cm-1

Term (1)4F is selected for further SOC calculation
    to change this, assign the variable "target" to another term in the table above


In [95]:
def term_energy_from_levels(df, term):
    # Given a DataFrame with the right columns ['J', 'termwt', Ecol],
    #   where 'Ecol' is the header for the column of level energies,
    # Return the term's average energy as derived from the levels
    global SOCI
    # find index for term 'term'
    iterm = SOCI.dfterm[SOCI.dfterm.Term == term].index[0]
    termwt = np.array([twt[iterm] for twt in df.termwt])
    degen = 2 * df.J.values + 1
    dweight = degen * termwt  # total weight, including degeneracies
    Eterm = np.dot(df[Ecol], dweight) / dweight.sum()
    return Eterm

In [96]:
# Use experimental level energies via eq. (2)
Eterm = term_energy_from_levels(dfdiff, target)
SOC2 = -Eterm
print('Applying eq. (2) (experimental energies and theoretical term weights)')
print(f'For term {target}, SOC2 = {SOC2:.1f} cm-1')

Applying eq. (2) (experimental energies and theoretical term weights)
For term (1)4F, SOC2 = -820.8 cm-1


In [97]:
print(f'Molpro source file: {fname}\n')
print(f'Alternative values for E_so[{target}] of atom {atom}:')
print('-' * 25)
print('{:12s} {:.1f} cm-1'.format('eq (1)', SOC1))
print('{:12s} {:.1f} cm-1'.format('raw theory', SOCraw))
print('{:12s} {:.1f} cm-1'.format('eq (2)', SOC2))
print('-' * 25)

Molpro source file: co_20Q24D_1710_ctzdk_fc.pro

Alternative values for E_so[(1)4F] of atom Co:
-------------------------
eq (1)       -793.1 cm-1
raw theory   -861.1 cm-1
eq (2)       -820.8 cm-1
-------------------------


In [98]:
def term_distrib(term, df):
    # return the weights (including 2J+1) of term in levels
    global SOCI
    itarget = SOCI.dfterm[SOCI.dfterm.Term == term].index[0]
    wt = [twt[itarget] for twt in df.termwt]  # without 2J+1 weighting
    wt = wt * (2*df.J + 1)
    return wt

In [99]:
# Term energy errors as inferred from all levels
dftermerr = pd.DataFrame(columns=['Term', 'wmean', 'wstds'])
termlist = []
wmean = []
wstds = []
# also consider unsigned (absolute value) errors
uwmean = []
uwstds = []
for term in set(dfdiff.uTerm):
    termlist.append(term)
    weights = term_distrib(term, dfdiff).values
    m, s = chem.weighted_mean(dfdiff.err, weights)
    wmean.append(m)
    wstds.append(s)
    uerr = np.abs(dfdiff.err.values)
    um, us = chem.weighted_mean(uerr, weights)
    uwmean.append(um)
    uwstds.append(us)
dftermerr['Term'] = termlist
dftermerr['wmean'] = wmean
dftermerr['wstds'] = wstds
dftermerr['uwmean'] = uwmean
dftermerr['uwstds'] = uwstds

if dftermerr.isnull().values.any():
    print('*** Some terms are missing ***')
    print('Try decreasing the energy maximum ("termcut")')
else:
    print('Errors in term energies (cm-1) as inferred from full level distribution')
    print('    (not only levels where leading)')
    # default order same as experimental terms
    dftermerr.Term = pd.Categorical(dftermerr.Term, xterms)
    dftermerr = dftermerr.sort_values('Term')
dftermerr.sort_values('uwmean').style.format(fmt)

Errors in term energies (cm-1) as inferred from full level distribution
    (not only levels where leading)


,Term,wmean,wstds,uwmean,uwstds
4,(1)4F,41.6,17.9,41.6,17.9
7,(1)4P,265.4,20.1,265.4,20.1
0,(2)4F,408.5,25.3,408.5,25.3
2,(1)2G,530.9,23.5,530.9,23.5
3,(1)2F,762.7,37.5,762.7,37.5
1,(2)4P,1098.0,59.2,1098.0,59.2
5,(1)2P,1293.0,451.4,1293.0,451.4
6,(1)2D,5161.1,512.6,5161.1,512.6


In [100]:
print(f'Distribution of term "{target}" among levels:')
thrsh = 1.e-6
dfdistrib = dfso.copy()
dfdistrib[target] = term_distrib(target, dfso)
# remove rows with negligible weights
dfdistrib = dfdistrib[dfdistrib[target] > thrsh]
dfdistrib.drop(['termwt', 'Composition'], axis=1, inplace=True)
display(dfdistrib.sort_values(target, ascending=False).style.format(fmt))
print(f'Total weight of {target} = {dfdistrib[target].sum():.3f}')

Distribution of term "(1)4F" among levels:


,Lead,J,Jlbl,Erel,Eshift,E,Nr,(1)4F
0,(1)4F,4.5,(1)4F_9/2,0.0,-859.9,-1392.560252,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",9.975615
1,(1)4F,3.5,(1)4F_7/2,851.9,-8.0,-1392.556371,"[11, 12, 13, 14, 15, 16, 17, 18]",7.995013
2,(1)4F,2.5,(1)4F_5/2,1474.4,614.5,-1392.553534,"[19, 20, 21, 22, 23, 24]",5.992055
3,(1)4F,1.5,(1)4F_3/2,1898.5,1038.5,-1392.551602,"[25, 26, 27, 28]",3.988509
15,(1)2G,4.5,(1)2G_9/2,17028.9,16168.9,-1392.482663,"[93, 94, 95, 96, 97, 98, 99, 100, 101, 102]",0.024385
21,(1)2D,1.5,(1)2D_3/2,22796.7,21936.7,-1392.456383,"[125, 126, 127, 128]",0.009782
20,(1)2D,2.5,(1)2D_5/2,21584.6,20724.7,-1392.461906,"[119, 120, 121, 122, 123, 124]",0.007821
17,(1)2G,3.5,(1)2G_7/2,17729.0,16869.1,-1392.479473,"[105, 106, 107, 108, 109, 110, 111, 112]",0.004957
18,(1)2P,1.5,(1)2P_3/2,19196.8,18336.9,-1392.472785,"[113, 114, 115, 116]",0.001670
10,(1)4P,2.5,(1)4P_5/2,14038.9,13179.0,-1392.496286,"[71, 72, 73, 74, 75, 76]",0.000060


Total weight of (1)4F = 28.000


In [101]:
# Don't attempt level-fitting by default
1/0

ZeroDivisionError: division by zero

## Fitting of term energies to experimental level energies

In [ ]:
def levels_from_term_energies(term_order, term_energies):
    # Install the term energies along the SOCI.matrix diagonal and rediagonalize
    #   'term_order' is array of term symbols
    #   'term_energies' is array of corresponding energies (cm-1)
    # Return nothing
    #global SOCI
    #   'SOCI' is a fullmatSOCI() object and is modified
    term_dict = dict(zip(term_order, term_energies))
    newdiag = SOCI.matrix.diagonal().copy()
    for ibs in range(len(newdiag)):
        j = SOCI.sob_ici[ibs]
        term = SOCI.mrci[j].Term
        # install the new energy for the term
        newdiag[ibs] = term_dict[term]
    # update the matrix
    SOCI.fill_diagonal(newdiag)
    SOCI.diagonalize(store=True, vectors=True, sortval=False)  # update the SOCI() object
    SOCI.assign_atomic_J(quiet=True)
    return

In [ ]:
def rmserr(dfcomp):
    # Given a matched-up comparison of expt and theory, return the
    #   rms error in level energies
    # Weighted with degeneracies
    rmse = np.dot(dfcomp.err**2, dfcomp.degen.astype(float))
    rmse /= dfcomp.degen.sum()
    rmse = np.sqrt(rmse)
    return rmse
def obj_fun(exc_terme):
    # Given only excited term energies (assuming ground=0)
    #   return the RMSE
    global term_order, dfexpt, SOCI
    terme = [0] + list(exc_terme)  # the fixed energy of the ground term is 0
    levels_from_term_energies(term_order, terme)
    dfcomp = match_expt_theory_simple(dfexpt, SOCI.dfso)
    # drop rows with NaN (no matching level in the calculation)
    dfcomp.dropna(axis=0, inplace=True)
    rmse = rmserr(dfcomp)
    return rmse

In [ ]:
# Create global 'term_order' 
term_order = dfterm.Term.values
term_energies = dfterm.ecm.values

In [ ]:
print('Initial comparison with expt, before fitting')
rmse0 = rmserr(dfdiff)
print(f'RMSE = {rmse0:.1f} cm-1')
display(dfdiff.style.format(fmt))

In [ ]:
def freport(xvec):
    # callback function to monitor minimization
    # this version provides more info, at slightly increased cost
    freport.counter += 1
    print(f'{freport.counter:5d}', end='')
    for x in xvec:
        print(f'   {x:.1f}', end='')
    fun = obj_fun(xvec)
    print(f'  RMSE = {fun:.1f}')
    return
freport.counter = 0

In [ ]:
# Minimize the RMSE (may take hours)
exc_terme = list(term_energies)[1:]  # only excited terms; assume ground term = 0 energy
result = scipy.optimize.minimize(obj_fun, exc_terme, method='Nelder-Mead', callback=freport)

In [ ]:
if result.success:
    print(f'Minimization complete in {result.nit} iterations with {result.nfev} evaluations')
    print(f'RMSE = {result.fun:.1f} after Nelder-Mead')
    rmse = result.fun
    print('Fitted levels')
    fit_terme = [0] + list(result.x)
    levels_from_term_energies(term_order, fit_terme)
    dfcomp = match_expt_theory_simple(dfexpt, SOCI.dfso)
    # drop rows with NaN (no matching level in the calculation)
    dfcomp.dropna(axis=0, inplace=True)
    rmse = rmserr(dfcomp)
    # Convert str values of J to float
    dfcomp['J'] = dfcomp.J.apply(chem.halves_to_float)
    display(dfcomp.style.format(fmt))
    print(f'RMSE = {rmse:.1f} cm-1 compared with initial RMSE = {rmse0:.1f}')
else:
    print('*** Failure ***')
    print(result)

In [ ]:
# Add to Term DataFrame
print('Fitted term energies; "Erel" is relative to the lowest level')
dfterm['fitted'] = np.nan
for term, terme in zip(term_order, fit_terme):
    dfterm.loc[dfterm.Term == term, 'fitted'] = terme
Eshift = SOCI.dfso.Eshift.min()  # energy of ground level relative to lowest term
dfterm['Erel'] = (dfterm.fitted - Eshift) # relative to ground level
dfterm.style.format(fmt)

In [ ]:
SOCfit = -dfterm.loc[dfterm.Term == target, 'Erel'].values[0]

In [ ]:
print(f'Molpro source file: {fname}\n')
print(f'Alternative values for E_so[{target}] of atom {atom}:')
print('-' * 25)
print('{:12s} {:.1f} cm-1'.format('eq (1)', SOC1))
print('{:12s} {:.1f} cm-1'.format('raw theory', SOCraw))
print('{:12s} {:.1f} cm-1'.format('eq (2)', SOC2))
print('{:12s} {:.1f} cm-1'.format('term fitting', SOCfit))
print('-' * 25)